## Title for now ##

A research group has set up a MineCraft server to collect data on how people play video games. Players' actions are recorded as they navigate through the world. The data includes players data as well as sessions data. In this report we aim to analyze the question: Can age and played hours predict whether a player is subscribed or not in players data?   

In the players data, there are 196 observations and 7 variables. The 7 variables are:

- experience (how experienced the player is, type chr)
- subscribe (if they are subscribed,type lgl)
- hashedEmail (player's emails, type chr)
- played_hours (number of hours the player played, type dbl)
- name (player's name, type chr)
- gender (gender of the player, type chr)
- age (age of the player, type dbl)